---

<h2 style="text-align: center;"><b>Word embeddings</b></h2>
<br> Присутствуют обучающие материалы из DLS

Библиотеки, которые нам понадобятся сегодня (на самом деле 2 первые из них вам тоже понадобятся впоследствии)

In [ ]:
% pip install --upgrade nltk gensim bokeh

**NLTK** (Natural Language Toolkit) -- библиотека с множеством функций для работы с естественным языком (например, лемматизация, токенизация и т.д.)

https://www.nltk.org

We wull use it for tokenization for now.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir_path = '/content/drive/MyDrive/Методы визуализации данных/lab5'

In [ ]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

quora = list(open(f"/{dir_path}/quora.txt"))
# посмотрите на 10-й вопрос из набора данных
print(tokenizer.tokenize(quora[10]))

['Which', 'brand', 'should', 'go', 'with', 'the', 'GTX', '960', 'graphic', 'card', ',', 'MSI', ',', 'Zotac', 'or', 'ASUS', '?']


Давайте выделим все тексты из quora.txt файл и получаем массив массивов токенов:

In [ ]:
quora_tokenized = [tokenizer.tokenize(line.lower()) for line in quora]

Существует множество различных моделей для обучения встраиванию слов (Word2Vec, GloVe, Fast Text и т.д.). А пока давайте попробуем обучить word2vec.

Библиотека gensim поможет нам в этом - она предоставляет модели для обучения Word2Vec и дальнейшего исследования векторов

In [ ]:
from gensim.models import Word2Vec
# определите модель (точно так же, как модель torch) с параметрами
# и обучим его на наших данных
model = Word2Vec(quora_tokenized, # data for model to train on
                # size=32,         # embedding vector size
                 min_count=5,     # consider words that occured at least 5 times
                 window=3).wv     # define context as a 3-word window around the target word

Now we have trained model and can play with it:

In [ ]:
# мы можем получить вектор для любого слова в нашем словаре
model.get_vector('tv')

array([ 1.530374  ,  1.2015975 , -0.33127838,  2.50062   ,  0.18569607,
        0.05476848, -2.5567837 ,  1.1204015 , -0.68836886,  0.30123693,
        0.9275709 ,  1.2380383 ,  0.78000194,  0.48610306,  0.2954524 ,
        1.1372147 , -0.5157053 , -0.786031  , -1.0101213 ,  0.5046111 ,
        1.6720979 , -0.07300232, -0.0130686 ,  1.0836154 ,  2.6313918 ,
        0.4453753 , -0.7970233 , -0.03077436,  0.8313297 ,  0.6680893 ,
        0.4422334 ,  0.3720875 ,  0.35496554, -0.781917  ,  1.980874  ,
       -0.5653157 , -1.0313752 , -1.3483632 , -0.6934263 , -2.7469277 ,
       -0.3993484 , -3.5727804 , -0.67679775,  0.23576266,  0.24046136,
       -1.7911662 ,  1.349383  , -0.4083153 ,  0.6881298 , -0.41589695,
        1.3480207 , -0.12436488, -1.421476  ,  0.23035876,  0.01500013,
        1.0919224 ,  0.6599213 , -1.0301235 ,  0.73065364, -0.06148681,
       -0.2345687 ,  1.9006784 , -1.0075661 , -0.45342997,  1.0859596 ,
       -0.9404421 ,  1.6178622 , -1.3458971 ,  1.570211  ,  1.09

In [ ]:
# и получите слова, векторы которых наиболее похожи на вектор данного слова
model.most_similar('tv')

[('television', 0.8248835802078247),
 ('netflix', 0.6790497899055481),
 ('film', 0.6522358655929565),
 ('lcd', 0.6238173246383667),
 ('hbo', 0.6144078969955444),
 ('broadway', 0.6132541298866272),
 ('dvd', 0.60565185546875),
 ('kapil', 0.6041522026062012),
 ('4k', 0.6035748720169067),
 ('serials', 0.5954263806343079)]

Cуществуют предварительно обученные модели или даже предварительно вычисленные коллекции векторов слов!

In [ ]:
import gensim.downloader as api
model = api.load('glove-twitter-100') # list of available models: https://github.com/RaRe-Technologies/gensim-data#models

[==================================================] 100.0% 387.1/387.1MB downloaded


In [ ]:
# who is a person who has money and related to coding, but is really stupid?
# (it's how a model trained on text corpus thinks, not me)
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

Также, используя gensim, мы можем загрузить предварительно обученные векторы из файла vectors. Например, мы можем загрузить предварительно подготовленные векторы Google word2vec::

http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/

In [ ]:
# from gensim.models import KeyedVectors
# model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

## Visualization

Итак, давайте визуализируем наши вложения в двумерном пространстве.

Что нам нужно, так это:
1. получите вложения для пакета слов из модели gensim
2. уменьшите размерность этих вложений со 100-мерных до 2-мерных векторов
3. нормализуйте эти векторы
4. нарисуйте это на плоскости!

Так что давайте сделаем это:

In [ ]:
n_words = 1000
words = sorted(model.key_to_index.keys(),
               key=lambda word: model.get_vecattr(word, "count"), # sort by number of word occurencies
               reverse=True)[:n_words]

print(words[::10])

['<user>', 'i', ')', 'no', 'la', 'o', '>', 'are', 'we', '♥', '_', 'now', '~', 'people', "'re", '>>', '[', 'q', 'je', 'never', 'please', 'c', 'hahaha', 'لا', 'then', 'itu', 'had', 'ah', 'esta', 'does', 'apa', 'les', 'work', 'god', 'ha', 'their', 'в', '\\', 'gue', '＾', 'justin', 'thing', 'hope', 'nigga', 'udah', 'same', 'voy', 'find', 'اللي', 'face', 'text', 'kau', 'very', 'may', 'pretty', 'already', 'qué', 'día', 'money', 'name', 'hari', 'wow', 'black', 'vamos', 'nao', 'head', 'trying', 'lot', 'ان', 'mad', 'playing', '✌', 'own', 'friday', 'estar', 'yet', 'une', 'mana', 'لي', 'car', 'once', 'nem', 'summer', 'fan', 'tudo', 'sexy', 'sea', 'excited', 'far', 'lost', 'sei', 'sabe', 'picture', 'dice', 'km', 'fast', 'moi', 'anda', 'noche', 'relationship']


In [ ]:
# get embeddings for those words
word_vectors = [model.get_vector(word) for word in words]

For vectors visualization we need to reduce dimensionality of vectors to 2 (or 3). We'll use **PCA** for that

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(word_vectors)
word_vectors_pca = pca.transform(word_vectors)

Also let's normalize vectors we got from PCA for better visibility

We already did normalization by hand in one of the first homeworks in our course, but for now let's use sklearn api:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(word_vectors_pca)
word_vectors_pca = ss.transform(word_vectors_pca)

Finally we are to draw an (interactive!) space of embeddings, using function below:

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig


draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

figure(id='p1004', ...)

Well, as we can see, this space does not look perfect, sometimes we see (it seems) different words grouped together.

That happend mostly because our way to reduce dimensionality of vectors (PCA)  is not perfect. Let's try to use different algorithm for doing this: TSNE.

TSNE is in some way creates embeddings itself, so we may hope that it will work better)

https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

https://distill.pub/2016/misread-tsne/

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

word_vectors = [model[word] for word in words]
word_vectors = np.array(word_vectors)

tsne = TSNE(n_components=2, verbose=50)
word_vectors_tsne = tsne.fit_transform(word_vectors)

ss = StandardScaler().fit(word_vectors_tsne)
word_vectors_tsne = ss.transform(word_vectors_tsne)

print(words[::10])

NameError: ignored

In [ ]:
output_notebook()
draw_vectors(word_vectors_tsne[:, 0], word_vectors_tsne[:, 1], token=words)

Задание

In [ ]:
with open(f"{dir_path}/dostoevsky.txt", 'r', encoding='utf-8-sig') as file:
    dostoevsky_text = file.read()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import string

nltk.download('punkt')

def preprocess_text(text):
    # Разбиваем текст на предложения
    sentences = sent_tokenize(text)

    # Инициализируем знаки препинания
    punctuation = set(string.punctuation)

    # Подготавливаем список токенов (слов) для каждого предложения
    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]

    # Удаляем знаки препинания
    cleaned_sentences = [
        [word.lower() for word in sentence if word not in punctuation]
        for sentence in tokenized_sentences
    ]

    # Подсчитываем количество слов и предложений
    # num_words = sum(len(sentence) for sentence in cleaned_sentences)
    # num_sentences = len(cleaned_sentences)

    return cleaned_sentences


cleaned_sentences = preprocess_text(dostoevsky_text)


print(f"Количество слов: {sum(len(sentence) for sentence in cleaned_sentences)}")
print(f"Количество предложений: {len(cleaned_sentences)}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Количество слов: 1883872
Количество предложений: 123550


In [ ]:
cleaned_sentences[0]

['федор',
 'михайлович',
 'достоевский',
 'бедные',
 'люди',
 'ох',
 'уж',
 'эти',
 'мне',
 'сказочники']

In [ ]:
%pip install navec

In [ ]:
from navec import Navec

path = f'{dir_path}/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
# Функция для создания векторных представлений
def get_embeddings(model, tokens):
    embeddings = []
    for token in tokens:
        if token in model:
            embeddings.append(model[token])

    return embeddings

In [ ]:
# Создание векторных представлений для униграмм
unigram_embeddings_navec = get_embeddings(navec,  (word for words in cleaned_sentences for word in words))

In [ ]:
text = "Это яблоко красное"

# Токенизация текста
tokens = text.lower().split()

# Создание векторных представлений для униграмм
unigram_embeddings = [navec[token] for token in tokens]

# Создание биграмм
bigrams = list(zip(tokens, tokens[1:]))

# Создание векторных представлений для биграмм
bigram_embeddings = [navec[bigram[0]] + navec[bigram[1]] for bigram in bigrams]

# Вывод результатов
print("Униграммы:")
for token, emb in zip(tokens, unigram_embeddings):
    print(f"{token}")

print("\nБиграммы:")
for bigram, emb in zip(bigrams, bigram_embeddings):
    print(f"{bigram}")

Униграммы:
это
яблоко
красное

Биграммы:
('это', 'яблоко')
('яблоко', 'красное')


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_words(word, model, top_n=3):
    if word not in model:
        return f"Слово '{word}' отсутствует в модели."

    word_vector = model[word]
    all_words = model.vocab.words
    all_vectors = np.array([model[w] for w in all_words])

    similarity_scores = cosine_similarity([word_vector], all_vectors)[0]
    similar_words_indices = similarity_scores.argsort()[-top_n - 1:-1][::-1]

    similar_words = [(all_words[i], similarity_scores[i]) for i in similar_words_indices]

    return similar_words

In [ ]:
find_similar_words("осень", navec, top_n=5)

[('весна', 0.774493),
 ('зима', 0.76086795),
 ('лето', 0.72997034),
 ('осени', 0.6942003),
 ('весну', 0.6692369)]

In [ ]:
find_similar_words("телевидение", navec, top_n=5)

[('радио', 0.6230979),
 ('телевидения', 0.5992099),
 ('кабельное', 0.5849143),
 ('сми', 0.577672),
 ('кинематограф', 0.551915)]

In [ ]:
n_words = 200
most_words_indexes = sorted(range(len(navec.vocab.words)), key=lambda i: navec.vocab.counts[i], reverse=True)[:n_words]

words = [navec.vocab.words[i] for i in most_words_indexes]

word_vectors = [navec.pq[i] for i in most_words_indexes]
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(word_vectors)
word_vectors_pca = pca.transform(word_vectors)

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig


draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

figure(id='p1004', ...)

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

word_vectors = np.array(word_vectors)

tsne = TSNE(n_components=2, verbose=50)
word_vectors_tsne = tsne.fit_transform(word_vectors)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(word_vectors_tsne)
word_vectors_tsne = ss.transform(word_vectors_tsne)

print(words[::10])

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 200 samples in 0.001s...
[t-SNE] Computed neighbors for 200 samples in 0.035s...
[t-SNE] Computed conditional probabilities for sample 200 / 200
[t-SNE] Mean sigma: 1.593408
[t-SNE] Computed conditional probabilities in 0.033s
[t-SNE] Iteration 50: error = 67.1594543, gradient norm = 0.4297648 (50 iterations in 7.751s)
[t-SNE] Iteration 100: error = 73.3877335, gradient norm = 0.4773179 (50 iterations in 2.979s)
[t-SNE] Iteration 150: error = 77.0337067, gradient norm = 0.4254592 (50 iterations in 3.249s)
[t-SNE] Iteration 200: error = 76.1586227, gradient norm = 0.3241881 (50 iterations in 2.961s)
[t-SNE] Iteration 250: error = 77.9973907, gradient norm = 0.4299411 (50 iterations in 3.812s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 77.997391
[t-SNE] Iteration 300: error = 1.1351578, gradient norm = 0.0086187 (50 iterations in 2.161s)
[t-SNE] Iteration 350: error = 1.0541279, gradient norm = 0.0077009 (

In [ ]:
draw_vectors(word_vectors_tsne[:, 0], word_vectors_tsne[:, 1], token=words)

figure(id='p1102', ...)